# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Ground-predictions" data-toc-modified-id="Ground-predictions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Ground predictions</a></div><div class="lev2 toc-item"><a href="#PVLib-Clearsky" data-toc-modified-id="PVLib-Clearsky-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>PVLib Clearsky</a></div><div class="lev1 toc-item"><a href="#Train/test-on-NSRDB-data" data-toc-modified-id="Train/test-on-NSRDB-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Train/test on NSRDB data</a></div>

In [1]:
import pandas as pd
import numpy as np
import os
import datetime
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn import tree

import pytz
import itertools
import visualize
import utils
import pydotplus
import xgboost as xgb

from sklearn import metrics

import pvlib
import cs_detection_refactor as cs_detection
# import cs_detection_refactor
# import visualize
# from bokeh.plotting import output_notebook
# output_notebook()

import visualize_plotly as visualize

from IPython.display import Image



%load_ext autoreload
%autoreload 2

np.set_printoptions(precision=4)
%matplotlib notebook

# Ground predictions

## PVLib Clearsky

Only making ground predictions using PVLib clearsky model and statistical model.  NSRDB model won't be available to ground measurements.

In [2]:
nsrdb = cs_detection.ClearskyDetection.read_pickle('abq_nsrdb_1.pkl.gz')
nsrdb.df.index = nsrdb.df.index.tz_convert('MST')

In [3]:
nsrdb.time_from_solar_noon('Clearsky GHI pvlib', 'tfn')

In [4]:
ground = cs_detection.ClearskyDetection.read_pickle('abq_ground_1.pkl.gz')
ground.df.index = ground.df.index.tz_convert('MST')

In [5]:
feature_cols = [
'tfn',
'abs_ideal_ratio_diff',
'abs_ideal_ratio_diff mean',
'abs_ideal_ratio_diff std',
'abs_ideal_ratio_diff max',
'abs_ideal_ratio_diff min',
# 'abs_ideal_ratio_diff range',
# 'abs_ideal_ratio_diff gradient',
# 'abs_ideal_ratio_diff gradient mean',
# 'abs_ideal_ratio_diff gradient std',
# 'abs_ideal_ratio_diff gradient max',
# 'abs_ideal_ratio_diff gradient min',
# 'abs_ideal_ratio_diff gradient range',
# 'abs_ideal_ratio_diff gradient second',
# 'abs_ideal_ratio_diff gradient second mean',
# 'abs_ideal_ratio_diff gradient second std',
# 'abs_ideal_ratio_diff gradient second max',
# 'abs_ideal_ratio_diff gradient second min',
# 'abs_ideal_ratio_diff gradient second range',
'GHI Clearsky GHI pvlib gradient ratio', 
'GHI Clearsky GHI pvlib gradient ratio mean', 
'GHI Clearsky GHI pvlib gradient ratio std', 
'GHI Clearsky GHI pvlib gradient ratio min', 
'GHI Clearsky GHI pvlib gradient ratio max', 
# 'GHI Clearsky GHI pvlib gradient ratio range', 
'GHI Clearsky GHI pvlib gradient second ratio', 
'GHI Clearsky GHI pvlib gradient second ratio mean', 
'GHI Clearsky GHI pvlib gradient second ratio std', 
'GHI Clearsky GHI pvlib gradient second ratio min', 
'GHI Clearsky GHI pvlib gradient second ratio max', 
# 'GHI Clearsky GHI pvlib gradient second ratio range',
'GHI Clearsky GHI pvlib line length ratio',
'GHI Clearsky GHI pvlib line length ratio gradient',
'GHI Clearsky GHI pvlib line length ratio gradient second'
]

target_cols = ['sky_status']

# Train/test on NSRDB data

In [8]:
train = cs_detection.ClearskyDetection(nsrdb.df)
train.scale_model('GHI', 'Clearsky GHI pvlib', 'sky_status')
train.calc_all_window_metrics(train.df, 3, col1='GHI', col2='Clearsky GHI pvlib', overwrite=True)

In [9]:
from sklearn import preprocessing

In [11]:
X = preprocessing.StandardScaler().fit_transform(train.df[feature_cols].values)

In [12]:
from sklearn import decomposition

In [26]:
pca = decomposition.PCA(n_components=3)

In [27]:
X_pca = pca.fit_transform(X)

In [28]:
pca.explained_variance_ratio_

array([ 0.2362,  0.1805,  0.174 ])

In [29]:
import plotly.plotly as py
from plotly.graph_objs import *
import plotly.tools as tls
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(True)

In [31]:
trace = Scatter3d(x=X_pca[:, 0], y=X_pca[:, 1], z=X_pca[:, 2], mode='markers')
plot([trace])

'file:///Users/benellis/duramat/clearsky_detection/temp-plot.html'